# Eliminação de Gauss

### Prof. Pedro Peixoto <ppeixoto@usp.br>

Abril 2022


In [1]:
import numpy as np
import sys
import time

### O problema Ax=b

In [75]:
# Tamanho do sistema
n = 300

def create_matrix(n):
    # Matriz aumentada A|b
    a = np.zeros((n,n+1))

    # Matriz A
    for i in range(n):
        for j in range(n):
            if (i == j) :
                a[i,j] = 2
            elif (abs(i-j)< 2):
                a[i,j] = -1

    # Vetor b 
    for i in range(n):
        a[i,n] = 1

    #print(a)
    return a
    

### O esquema clássico, sem pivotamento

In [77]:
a = create_matrix(n)
x = np.zeros(n)
A = a[:,:-1]
b = a[:,n]

start_time = time.time()
# Escalonamento
for i in range(n):
    if a[i,i] == 0.0:
        sys.exit('Ainda não implementei pivotamento :-( ')
        
    for j in range(i+1, n):
        ratio = a[j,i]/a[i,i]
        for k in range(n+1):
            a[j,k] = a[j,k] - ratio * a[i,k]

# Substituição
x[n-1] = a[n-1,n]/a[n-1,n-1]

for i in range(n-2,-1,-1):
    x[i] = a[i,n]
    
    for j in range(i+1,n):
        x[i] = x[i] - a[i,j]*x[j]
    
    x[i] = x[i]/a[i,i]
tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(A@x- b)))


Soma da Solução: 2272550.0000000903  Tempo que levou:  4.813074111938477

Solução: 
150.00000000000156

Teste: 
1.9895196601282805e-12


### Dá para vetorizar?

In [78]:
a = create_matrix(n)
x = np.zeros(n)
A = a[:,:-1]
b = a[:,n]

start_time = time.time()

# Escalonamento
for i in range(n):
    if a[i,i] == 0.0:
        sys.exit('Ainda não implementei pivotamento :-( ')
        
    for j in range(i+1, n):        
        #Vetorizei aqui!
        a[j,:] = a[j,:] - a[j,i]/a[i,i] * a[i,:]

# Substituição
x[n-1] = a[n-1,n]/a[n-1,n-1]

for i in range(n-2,-1,-1):
    #Vetorizei aqui!
    x[i] = (a[i,n] - np.dot(a[i,i+1:n], x[i+1:n]))/a[i,i]

tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(A@x- b)))

Soma da Solução: 2272550.0000000903  Tempo que levou:  0.12741398811340332

Solução: 
150.00000000000156

Teste: 
1.9895196601282805e-12


### Dá para vetorizar mais?

Para isso tem que mudar o algoritmos, vejamos um algoritmo de decomposição LU (que vai serir de base para Elim. Gaussiana vetorial): Algoritmo de Crout (ou Doolittle, dependendo de onde colocamos a diagonal unitária).


In [79]:
a = create_matrix(n)

A = a[:,:-1]
b = a[:,n]

start_time = time.time()

#Guardo tanto L quanto U em uma única matriz!!!!
LU = np.eye(n)
# @ faz multiplicação de matrizes usando numpy
for i in range(n):
    #Varre linhas superiores (Upper)
    LU[i,i:] = A[i,i:]-LU[i,:i] @ LU[:i,i:]
    #Varre colunas inferiores (Lower)
    LU[(i+1):,i] = ( A[(i+1):,i]- LU[(i+1):,:i] @ LU[:i,i] ) / LU[i,i]


# Substituição
# LUx=b
# Ly=b
y = np.zeros(n)
# Ly=b
y[0] = b[0]
for i in range(1,n,1):
    #Vetorizei aqui!
    y[i] = (b[i] - np.dot(LU[i,:i], y[:i]))

# Ux=y
x = np.zeros(n)
x[n-1] = y[n-1]/LU[n-1,n-1]
for i in range(n-2,-1,-1):
    #Vetorizei aqui!
    x[i] = (y[i] - np.dot(LU[i,i+1:], x[i+1:]))/LU[i,i]

tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(A@x- b)))

#Testes
U = np.triu(LU)
L = np.tril(LU)
np.fill_diagonal(L, 1.0)
print('\nTeste LU: ')
print(np.max(np.max(np.abs(L@U-A))))

Soma da Solução: 2272550.0000000903  Tempo que levou:  0.025944948196411133

Solução: 
150.00000000000156

Teste: 
3.637978807091713e-12

Teste LU: 
1.1102230246251565e-16


Desafio: Você consegue usar essa ideia do LU para fazer diretamente a solução do sistema linear vetorial, com apenas um loop, sem precisar calcular L e U explicitamente como fizemos?